In [1]:
import pandas as pd
import numpy as np

from skopt import gp_minimize, gbrt_minimize
from skopt.space import Integer
from skopt.utils import use_named_args
from neuralforecast import NeuralForecast
from neuralforecast.models import RNN
from neuralforecast.losses.pytorch import MQLoss, MAE, MSE
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
import warnings
warnings.filterwarnings('ignore')

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
df=pd.read_csv('vnindex_with_usd_dji_gold_oil.csv')
df=df[['date','OT']]

df['date'] = pd.to_datetime(df['date'])
df

,date,OT
0,2015-01-05,544.45
1,2015-01-06,549.66
2,2015-01-07,552.05
3,2015-01-08,553.47
4,2015-01-09,569.73
...,...,...
2551,2025-03-27,1323.81
2552,2025-03-28,1317.46
2553,2025-03-31,1306.86
2554,2025-04-01,1317.33


In [3]:
df1 = df
df1 = df1.reset_index().rename(columns={'date': 'ds'})
df1['unique_id'] = 'VNINDEX'
df1.rename(columns={'OT': 'y'}, inplace=True)
df1['ds'] = pd.to_datetime(df1['ds'])
# Kết quả
df1 = df1[['unique_id', 'ds', 'y']]
df1

,unique_id,ds,y
0,VNINDEX,2015-01-05,544.45
1,VNINDEX,2015-01-06,549.66
2,VNINDEX,2015-01-07,552.05
3,VNINDEX,2015-01-08,553.47
4,VNINDEX,2015-01-09,569.73
...,...,...,...
2551,VNINDEX,2025-03-27,1323.81
2552,VNINDEX,2025-03-28,1317.46
2553,VNINDEX,2025-03-31,1306.86
2554,VNINDEX,2025-04-01,1317.33


In [4]:
# Xác định kích thước các tập
train_size = int(len(df1) * 0.7)  # 70% cho train
val_len = int(len(df1) * 0.2)   # 20% cho validation
test_len = len(df1) - train_size - val_len  # Phần còn lại cho test

# Chia dữ liệu theo thời gian
train = df1.iloc[:train_size].reset_index(drop=True)
val = df1.iloc[train_size:train_size + val_len].reset_index(drop=True)
test = df1.iloc[train_size + val_len:].reset_index(drop=True)

# Kiểm tra kích thước
print("VNINDEX:")
print(f"Train size: {len(train)}")
print(f"Validation size: {len(val)}")
print(f"Test size: {len(test)}")


VNINDEX:
Train size: 1789
Validation size: 511
Test size: 256


# Bayes

In [9]:
### Bayes
import pandas as pd
import numpy as np
from skopt import gp_minimize, gbrt_minimize
from skopt.space import Integer,Categorical,Real
from skopt.utils import use_named_args
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import DistributionLoss, MAE, MSE
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape


In [13]:


# Giả định df, val_len, test_len đã được định nghĩa trước
horizon = 20  


# Định nghĩa không gian tìm kiếm cho các siêu tham số LSTM
dimensions = [
    Integer(low=40, high=80, name='input_size'),  # Bắt đầu từ mẫu: input_size=8
    Integer(low=100, high=500, name='max_steps'),  # Bắt đầu từ mẫu: max_steps=200
    Integer(low=1, high=3, name='encoder_n_layers'),  # Bắt đầu từ mẫu: encoder_n_layers=2
    Categorical(categories=[32, 64], name='batch_size'),  # Bắt đầu từ mẫu: encoder_hidden_size=128
    Integer(low=1, high=3, name='decoder_layers'),  # Bắt đầu từ mẫu: decoder_layers=2
    Real(low=1e-5, high=1e-3, prior='log-uniform', name='learning_rate'),
]

# Biến toàn cục để lưu kết quả
results_history = []
best_score = float('inf')
best_params = None
best_method = None
best_result = None

# Hàm callback để lưu kết quả tối ưu hóa
def callback_func(result):
    global results_history, best_score, best_params, best_method, best_result
    
    current_score = result.func_vals[-1]
    current_params = dict(zip([dim.name for dim in dimensions], result.x_iters[-1]))
    
    results_history.append({
        'iteration': len(result.func_vals),
        'score': current_score,
        **current_params
    })
    
    if current_score < best_score:
        best_score = current_score
        best_params = current_params.copy()
        best_result = result
        print(f"Điểm tốt nhất mới: {current_score:.4f} tại vòng lặp {len(result.func_vals)}")

@use_named_args(dimensions)
def objective(**params):
    """
    Hàm mục tiêu cho Tối ưu hóa Bayes
    """
    try:
    
        
        # Tạo mô hình LSTM với các siêu tham số
        model = LSTM(
            h=horizon,
            input_size=params['input_size'],
            loss=DistributionLoss(distribution="Normal", level=[80, 90]),
            scaler_type='robust',
            encoder_n_layers=params['encoder_n_layers'],
            decoder_layers=params['decoder_layers'],
            batch_size=params['batch_size'],
            learning_rate=params['learning_rate'],
            max_steps=params['max_steps'],
            recurrent=True,
            h_train=1,
            valid_loss=DistributionLoss(distribution="Normal", level=[80, 90])
        )
        
        # Khởi tạo NeuralForecast
        fcst = NeuralForecast(models=[model], freq='B')
        
        # Thực hiện cross-validation
        print("Hình dạng dữ liệu đầu vào:", df1.shape)
        forecasts = fcst.cross_validation(
            df=df1, 
            val_size=val_len, 
            n_windows=None, 
            test_size=test_len - ((test_len - horizon) % horizon), 
            step_size=horizon
        )
        
        # Tính toán MAE
        Y_true = forecasts['y'].values
        Y_hat = forecasts['LSTM'].values
        print("Hình dạng dự báo:", forecasts.shape)
        MAE_score = mae(Y_true, Y_hat)
        
        print(f"MAE: {MAE_score:.4f} | Tham số: {params}")
        return MAE_score
        
    except Exception as e:
        print(f"Lỗi trong hàm mục tiêu: {e}")
        return 1000.0  # Phạt cho các lần chạy thất bại



In [14]:

# Phương pháp 1: Tối ưu hóa Gaussian Process
print("="*60)
print("PHƯƠNG PHÁP 1: TỐI ƯU HÓA GAUSSIAN PROCESS")
print("="*60)

results_history = []  # Đặt lại lịch sử
result_gp = gp_minimize(
    func=objective,
    dimensions=dimensions,
    n_calls=25,
    n_initial_points=10,
    acq_func='EI',
    random_state=42,
    callback=callback_func
)

print(f"Điểm GP tốt nhất: {result_gp.fun:.4f}")
print("Tham số GP tốt nhất:")
for dim, val in zip(dimensions, result_gp.x):
    print(f"  {dim.name}: {val}")
if result_gp.fun < best_score:
    best_method = "Gaussian Process"
    best_result = result_gp

# Phương pháp 2: Tối ưu hóa Gradient Boosted Trees
print("\n" + "="*60)
print("PHƯƠNG PHÁP 2: TỐI ƯU HÓA GRADIENT BOOSTED TREES")
print("="*60)

results_history = []  # Đặt lại lịch sử
result_gbrt = gbrt_minimize(
    func=objective,
    dimensions=dimensions,
    n_calls=25,
    n_initial_points=10,
    random_state=42,
    callback=callback_func
)

print(f"Điểm GBRT tốt nhất: {result_gbrt.fun:.4f}")
print("Tham số GBRT tốt nhất:")
for dim, val in zip(dimensions, result_gbrt.x):
    print(f"  {dim.name}: {val}")
if result_gbrt.fun < best_score:
    best_method = "Gradient Boosted Trees"
    best_result = result_gbrt


Seed set to 1


PHƯƠNG PHÁP 1: TỐI ƯU HÓA GAUSSIAN PROCESS


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Hình dạng dữ liệu đầu vào: (2556, 3)



  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 5      | train
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | hist_encoder | LSTM             | 9.5 K  | train
----------------------------------------------------------
9.5 K     Trainable params
5         Non-trainable params
9.5 K     Total params
0.038     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


...

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Hình dạng dữ liệu đầu vào: (2556, 3)



  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 5      | train
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | hist_encoder | LSTM             | 6.1 K  | train
----------------------------------------------------------
6.1 K     Trainable params
5         Non-trainable params
6.1 K     Total params
0.025     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Lỗi trong hàm mục tiêu: batch_size should be a positive integer value, but got batch_size=32
Điểm GBRT tốt nhất: 23.6737
Tham số GBRT tốt nhất:
  input_size: 41
  max_steps: 464
  encoder_n_layers: 2
  batch_size: 64
  decoder_layers: 2
  learning_rate: 0.00010968217207529519


In [15]:

# So sánh kết quả các phương pháp
print("\n" + "="*60)
print("SO SÁNH KẾT QUẢ")
print("="*60)

methods_comparison = pd.DataFrame({
    'Phương pháp': ['Gaussian Process', 'Gradient Boosted Trees'],
    'MAE tốt nhất': [result_gp.fun, result_gbrt.fun],
    'Số lần gọi': [len(result_gp.func_vals), len(result_gbrt.func_vals)]
})

print(methods_comparison.to_string(index=False))



SO SÁNH KẾT QUẢ
           Phương pháp  MAE tốt nhất  Số lần gọi
      Gaussian Process     24.182323          25
Gradient Boosted Trees     23.673719          25


In [18]:

# Huấn luyện mô hình cuối cùng với tham số tốt nhất
print("\n" + "="*60)
print("HUẤN LUYỆN MÔ HÌNH CUỐI CÙNG")
print("="*60)

final_params = dict(zip([dim.name for dim in dimensions], best_result.x))


# final_model = LSTM(
#     h=horizon,
#     input_size=final_params['input_size'],
#     loss=DistributionLoss(distribution="Normal", level=[80,90]),
#     scaler_type='robust',
#     encoder_n_layers=final_params['encoder_n_layers'],
#     encoder_hidden_size=final_params['encoder_hidden_size'],
#     decoder_layers=final_params['decoder_layers'],
#     decoder_hidden=final_params['decoder_hidden_size'],
#     max_steps=final_params['max_steps'],
#     recurrent=True,
#     h_train=1.0,
#     valid_loss=DistributionLoss(distribution='Normal', level=[80, 90])
# )
final_model = LSTM(
            h=horizon,
            input_size=final_params['input_size'],
            loss=DistributionLoss(distribution="Normal", level=[80, 90]),
            scaler_type='robust',
            encoder_n_layers=final_params['encoder_n_layers'],
            decoder_layers=final_params['decoder_layers'],
            batch_size=final_params['batch_size'],
            learning_rate=final_params['learning_rate'],
            max_steps=final_params['max_steps'],
            recurrent=True,
            h_train=1,
            valid_loss=DistributionLoss(distribution="Normal", level=[80, 90])
        )

fcst_final = NeuralForecast(models=[final_model], freq='B')
forecasts_final = fcst_final.cross_validation(
    df=df1, 
    val_size=val_len, 
    n_windows=None, 
    test_size=test_len - ((test_len - horizon) % horizon), 
    step_size=horizon
)

# Tính toán các chỉ số cuối cùng
Y_true_final = forecasts_final['y'].values
Y_hat_final = forecasts_final['LSTM'].values

MAE_final = mae(Y_true_final, Y_hat_final)
MAPE_final = mape(Y_true_final, Y_hat_final) * 100
RMSE_final = np.sqrt(mse(Y_true_final, Y_hat_final))

# Kết quả cuối cùng
df_final_results = pd.DataFrame({
    'Chân trời': [str(horizon)],
    'Dữ liệu': ["VNINDEX"],
    'MAE': [MAE_final],
    'RMSE': [RMSE_final],
    'MAPE': [MAPE_final]
})

print("\nKẾT QUẢ CUỐI CÙNG:")
print(df_final_results.to_string(index=False))

# Lưu kết quả
results_df = pd.DataFrame({
    'GP_MAE': result_gp.func_vals,
    'GBRT_MAE': result_gbrt.func_vals
})

# Tóm tắt
print("\n" + "="*60)
print("TÓM TẮT")
print("="*60)
print(f"Phương pháp tốt nhất: {best_method}")
print(f"MAE tốt nhất: {best_result.fun:.4f}")
print(f"Số lần gọi hàm: {len(best_result.func_vals)}")
print("\nTham số tối ưu:")
for dim, val in zip(dimensions, best_result.x):
    print(f"  {dim.name}: {val}")

Seed set to 1


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 5      | train
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | hist_encoder | LSTM             | 6.1 K  | train
----------------------------------------------------------
6.1 K     Trainable params
5         Non-trainable params
6.1 K     Total params
0.025     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode



HUẤN LUYỆN MÔ HÌNH CUỐI CÙNG


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=464` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]


KẾT QUẢ CUỐI CÙNG:
Chân trời Dữ liệu       MAE      RMSE     MAPE
       20 VNINDEX 23.673719 31.572596 1.890625

TÓM TẮT
Phương pháp tốt nhất: None
MAE tốt nhất: 23.6737
Số lần gọi hàm: 8

Tham số tối ưu:
  input_size: 41
  max_steps: 464
  encoder_n_layers: 2
  batch_size: 64
  decoder_layers: 2
  learning_rate: 0.00010968217207529519


In [23]:
MAE_final = mae(Y_true_final, Y_hat_final)
MAPE_final = mape(Y_true_final, Y_hat_final) * 100
RMSE_final = np.sqrt(mse(Y_true_final, Y_hat_final))

# Kết quả cuối cùng
df_final_results = pd.DataFrame({
    'Chân trời': [str(horizon)],
    'Dữ liệu': ["VNINDEX"],
    'MAE': [MAE_final],
    'RMSE': [RMSE_final],
    'MAPE': [MAPE_final]
})

df_final_results

,Chân trời,Dữ liệu,MAE,RMSE,MAPE
0,20,VNINDEX,23.673719,31.572596,1.890625
